In [1]:
%load_ext sparkmagic.magics

http://192.168.1.123:8998/

In [2]:
%manage_spark

TWFnaWNzQ29udHJvbGxlcldpZGdldChjaGlsZHJlbj0oVGFiKGNoaWxkcmVuPShNYW5hZ2VTZXNzaW9uV2lkZ2V0KGNoaWxkcmVuPShIVE1MKHZhbHVlPXUnPGJyLz4nKSwgSFRNTCh2YWx1ZT3igKY=


Added endpoint http://192.168.1.123:8998/
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,None,spark,idle,,,✔


SparkSession available as 'spark'.


In [3]:
%%spark
val sqlContext = new org.apache.spark.sql.hive.HiveContext(sc)

val df = sqlContext.read.format("com.crealytics.spark.excel").option("sheetName", "pc").option("useHeader", "true").option("inferSchema", "false").load("/home/churtado/notebooks/data/smmt.xlsx")
df.createTempView("pc")

sqlContext: org.apache.spark.sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@4abeecab
df: org.apache.spark.sql.DataFrame = [date: string, passenger_cars: string]


In [4]:
%%spark -c sql
SHOW TABLES

,database,tableName,isTemporary
0,default,cleaned_taxes,False
1,,pc,True


In [5]:
%%spark -c sql
SELECT * from pc limit 3

,date,passenger_cars
0,2018-06-30,234945
1,2018-05-31,192649
2,2018-04-30,167911


In [10]:
%%spark -c sql
DROP TABLE IF EXISTS passenger_cars

""


In [12]:
%%spark -c sql 

CREATE TABLE passenger_cars AS
SELECT 
 date, 
 passenger_cars
FROM pc

An error was encountered:
org.apache.spark.sql.AnalysisException: `default`.`passenger_cars` already exists.;
  at org.apache.spark.sql.hive.execution.CreateHiveTableAsSelectCommand.run(CreateHiveTableAsSelectCommand.scala:53)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:115)
  at org.apache.spark.sql.Dataset$$anonfun$6.apply(Dataset.scala:190)
  at org.apache.spark.sql.Dataset$$anonfun$6.apply(Dataset.scala:190)
  at org.apache.spark.sql.Dataset$$anonfun$52.apply(Dataset.scala:3254)
  at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:77)
  at org.apache.spark.sql.Dataset.withAction(Dataset.scala:3253)
  at org.apache.spark.sql.Dataset.<init>(Dataset.scala:190)
  at org.apache.spark.s

In [14]:
%%spark -c sql -q -o df_passenger_cars
SELECT * from pc

In [15]:
from autovizwidget.widget.utils import display_dataframe
display_dataframe(df_passenger_cars)

VkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oSFRNTCh2YWx1ZT11J1R5cGU6JyksIEJ1dHRvbihkZXNjcmlwdGlvbj11J1RhYmxlJywgbGF5b3V0PUxheW91dCh3aWR0aD11JzcwcHgnKSzigKY=


Output()

AutoVizWidget()

  
We're going to try and do a linear regression on this
  